<a href="https://colab.research.google.com/github/HausReport/ClubRaiders/blob/master/notebooks/History.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

!{sys.executable} -m pip install --force-reinstall --upgrade "git+https://github.com/HausReport/ClubRaiders.git#egg=craid"
import craid.eddb.loader.DataProducer as dp
import pandas as pd
import logging
import plotly.graph_objs as go
import plotly.express as px
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import timedelta, date


logging.getLogger().addHandler(logging.StreamHandler())
logging.getLogger().level = logging.DEBUG

arrays = dp.getDataArrays()
baseFrame: pd.DataFrame = arrays['dataFrame']

  Cloning https://github.com/HausReport/ClubRaiders.git to /tmp/pip-install-9tsch988/craid
  Running command git clone -q https://github.com/HausReport/ClubRaiders.git /tmp/pip-install-9tsch988/craid
     |████████████████████████████████| 71kB 4.9MB/s 
     |████████████████████████████████| 3.4MB 12.5MB/s 
     |████████████████████████████████| 194kB 46.7MB/s 
     |████████████████████████████████| 1.8MB 45.2MB/s 
     |████████████████████████████████| 1.2MB 41.0MB/s 
     |████████████████████████████████| 102kB 12.5MB/s 
     |████████████████████████████████| 10.5MB 48.1MB/s 
     |████████████████████████████████| 71kB 9.2MB/s 
     |████████████████████████████████| 798kB 45.7MB/s 
     |████████████████████████████████| 14.5MB 315kB/s 
     |████████████████████████████████| 235kB 48.2MB/s 
     |████████████████████████████████| 184kB 50.1MB/s 
     |████████████████████████████████| 133kB 51.1MB/s 
     |████████████████████████████████| 460kB 40.5MB/s 
     |█████████████

Acquired lock.
Loading from AWS
Memory tracker: 0:211,533,824
1- downloading to: /tmp/smol-factions.jsonl.gz
2 - downloading [https://erlaed.s3.us-east-2.amazonaws.com/smol-factions.jsonl.gz] to [/tmp/smol-factions.jsonl.gz] data file.
Starting new HTTPS connection (1): erlaed.s3.us-east-2.amazonaws.com:443
https://erlaed.s3.us-east-2.amazonaws.com:443 "GET /smol-factions.jsonl.gz HTTP/1.1" 200 15266
Found data file: /tmp/smol-factions.jsonl.gz
Read 654 lines of faction data
Memory tracker: 0.5:213,114,880
1- downloading to: /tmp/smol-systems_populated.jsonl.gz
2 - downloading [https://erlaed.s3.us-east-2.amazonaws.com/smol-systems_populated.jsonl.gz] to [/tmp/smol-systems_populated.jsonl.gz] data file.
Starting new HTTPS connection (1): erlaed.s3.us-east-2.amazonaws.com:443
https://erlaed.s3.us-east-2.amazonaws.com:443 "GET /smol-systems_populated.jsonl.gz HTTP/1.1" 200 25348
Found data file: /tmp/smol-systems_populated.jsonl.gz
Read 169 lines of systems data
Memory tracker: 1:215,535

In [80]:
#pip install "pandas-profiling"


     |████████████████████████████████| 266kB 8.0MB/s 
  Using cached https://files.pythonhosted.org/packages/30/9e/f663a2aa66a09d838042ae1a2c5659828bb9b41ea3a6efa20a20fd92b121/Jinja2-2.11.2-py2.py3-none-any.whl
     |████████████████████████████████| 3.1MB 13.5MB/s 
     |████████████████████████████████| 6.5MB 17.8MB/s 
  Using cached https://files.pythonhosted.org/packages/a7/f7/2adca20a7fa71b6a32f823bbd83992adeceab1d8bf72992bb7a55c69c19a/pandas-1.1.0-cp36-cp36m-manylinux1_x86_64.whl
     |████████████████████████████████| 122kB 49.6MB/s 
     |████████████████████████████████| 11.5MB 51.3MB/s 
     |████████████████████████████████| 307kB 42.5MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
  Using cached https://files.pythonhosted.org/packages/b1/9a/7d474ba0860a41f771c9523d8c4ea56b084840b5ca4092d96bdee8a3b684/numpy-1.19.1-cp36-cp36m-manylinux2010_x86_64.whl
     |████████████████████████████████| 71kB 10.2MB/s 
     |████████████████████████████████| 25.9MB 1.4MB/s 
   

In [2]:
from craid.eddb.loader.strategy.DataLoader import DataLoader
from craid.eddb.loader.strategy.GithubLoader import LoadDataFromGithub
from craid.history.History import History

myLoader = LoadDataFromGithub(_forceWebDownload=True, useSmol=False)
hist = History()
csa = hist.getRawDataFrame()
target = hist.getNormalizedDataFrame()

1- downloading to: /tmp/history.jsonl.gz
2 - downloading [https://erlaed.s3.us-east-2.amazonaws.com/history.jsonl.gz] to [/tmp/history.jsonl.gz] data file.
Starting new HTTPS connection (1): erlaed.s3.us-east-2.amazonaws.com:443


Creating the object


https://erlaed.s3.us-east-2.amazonaws.com:443 "GET /history.jsonl.gz HTTP/1.1" 200 120416
Found data file: /tmp/history.jsonl.gz
Read system        10340
faction       10340
updated       10340
influence     10340
control       10340
region        10340
population    10340
dtype: int64 lines of history data


In [3]:
def getScroller(fig):
  fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
  ))

In [4]:
def getFactionInfluenceGraph(fac: str):
  ogun = target[ target['faction']==fac]
  a = ogun['system'].unique().tolist()
  fig = go.Figure()
  for sys in a:
    theSlice = ogun[ ogun['system'] == sys].reset_index()
    fig.add_trace(go.Scatter(x=theSlice['updated'], y=theSlice['influence'],
                    mode='lines',
                    stackgroup="b",
                    name=sys))
  getScroller(fig)
  return fig

In [5]:
def getSystemInfluenceGraph(sys: str):
  ogun = target[ target['system']==sys]
  a = ogun['faction'].unique().tolist()
  fig = go.Figure()
  for fac in a:
    theSlice = ogun[ ogun['faction'] == fac].reset_index()
    fig.add_trace(go.Scatter(x=theSlice['updated'], y=theSlice['influence'],
                    mode='lines',
                    name=fac))
  getScroller(fig)
  return fig

In [6]:
# data cleaning
# done in api now
#csa = csa.drop_duplicates()
#csa = csa[ csa['faction'] != 'Aegis of Federal Democrats']
#csa = csa[ csa['faction'] != 'Aegis Imperium']
#csa = csa[ csa['faction'] != "Emperor's Dawn"]

In [7]:
stage1 = csa.loc[csa.groupby(['faction','system']).updated.idxmax()]
stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
fig = px.pie(stage3, values='influence', names='faction', title='Total Influence by Minor Faction', width=350, height=350)
fig.update_traces(textposition="inside")
fig.update_layout(showlegend=False)
fig.show()

In [8]:
from craid.club.regions.RegionFactory import RegionFactory
stage3 = stage1[['region','influence']].groupby('region').influence.sum().reset_index()
regionNames = list(RegionFactory.regionDict.keys())
fig = px.pie(stage3, values='influence', names=regionNames, title='Total Club Influence by Region', width=350, height=350) #, names='faction'
fig.update_traces(textposition="inside")
fig.update_layout(showlegend=False)
fig.show()

In [9]:
region = 1
regionNames = list(RegionFactory.regionDict.keys())
regionName = regionNames[region]
title = f"Total Influence <br>by Minor Faction <br>in the {regionName} region"
merope = target[ target['region']== region]
stage1 = merope.loc[merope.groupby(['faction','system']).updated.idxmax()]
stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
fig = px.pie(stage3, values='influence', names='faction', title=title, width=350, height=350) #, names='faction'
fig.update_traces(textposition="inside")
fig.update_layout(showlegend=False)
fig.show()

In [10]:
theDate = '2020-06-1'
#reg = csa[ csa['region']== 1]
dated = csa [ csa['updated']<= theDate]

stage1 = dated.loc[dated.groupby(['faction','system']).updated.idxmax()]
#stage1[['system','faction','influence']]
stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
stage3.influence.sum()

4168.610099040681

In [11]:
region = 8

dates = []
inf = []

start_date = date(2018, 5, 15)
daily_date = date(2020,5,30)
end_date = date.today() + timedelta(days=1) 

dframe = csa[ csa['region'] == region]
regionNames = list(RegionFactory.regionDict.keys())
regionName = regionNames[region]
title = f"Total Club Influence in the {regionName} region"

single_date = start_date
while single_date <= end_date:
    theDate = single_date.strftime("%Y-%m-%d")
    dated = dframe [ dframe['updated']<= theDate]

    stage1 = dated.loc[dated.groupby(['faction','system']).updated.idxmax()]
    stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
    theInf = stage3.influence.sum()
    dates.append(theDate)
    inf.append(theInf)
    if(single_date<daily_date):
      single_date = single_date + relativedelta(months=1)
    else:
      single_date = single_date + relativedelta(days=1)
      
labels = {'x':'Day', 'y':"Total Influence"}
fig = px.line(x=dates, y=inf, title=title, labels=labels)
fig.show()

In [12]:

dates = []
inf = []

start_date = date(2018, 5, 15)
daily_date = date(2020,5,30)
end_date = date.today() + timedelta(days=1) 

dframe = csa
regionNames = list(RegionFactory.regionDict.keys())
regionName = regionNames[region]
title = f"Total Club Influence"

single_date = start_date
while single_date <= end_date:
    theDate = single_date.strftime("%Y-%m-%d")
    dated = dframe [ dframe['updated']<= theDate]

    stage1 = dated.loc[dated.groupby(['faction','system']).updated.idxmax()]
    stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
    theInf = stage3.influence.sum()
    dates.append(theDate)
    inf.append(theInf)
    if(single_date<daily_date):
      single_date = single_date + relativedelta(months=1)
    else:
      single_date = single_date + relativedelta(days=1)

labels = {'x':'Day', 'y':"Total Influence"}
fig = px.line(x=dates, y=inf, title=title, labels=labels)
fig.show()

In [13]:
dates = []
inf = []

faction = "Sirius Corporation"
faction = "Hodack Prison Colony"
faction = "Wreaken Construction"
#faction = "'Sirius Drives'"
faction = "Aegis Core"
#faction = "Aegis Research"
#faction = "Bill Turner"

start_date = date(2018, 5, 15)
daily_date = date(2020,5,30)
end_date = date.today() + timedelta(days=1) #date(2020, 6, 28)


dframe = csa[ csa['faction'] == faction]
regionNames = list(RegionFactory.regionDict.keys())
regionName = regionNames[region]
title = f"Total Influence of the {faction} Faction"

single_date = start_date
while single_date <= end_date:
    theDate = single_date.strftime("%Y-%m-%d")
    dated = dframe [ dframe['updated']<= theDate]

    stage1 = dated.loc[dated.groupby(['faction','system']).updated.idxmax()]
    stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
    theInf = stage3.influence.sum()
    dates.append(theDate)
    inf.append(theInf)
    if(single_date<daily_date):
      single_date = single_date + relativedelta(months=1)
    else:
      single_date = single_date + relativedelta(days=1)


labels = {'x':'Day', 'y':"Total Influence"}
fig = px.line(x=dates, y=inf, title=title, labels=labels)
fig.show()

In [14]:
getFactionInfluenceGraph("Aegis Core").show()

In [15]:
badGuys = ['Abroin Universal PLC', 'Aegis Core', 'Aegis Research',
       "Benton's Gang", 'Bentonian Party', 'CQC Holdings',
       'Gallant Investment Brokers', 'Hodack Prison Colony',
       'Janus Incorporated', "Namarii Emperor's Dawn", 'Reyan BPS',
       'Reynhardt IntelliSys', 'Sirius Atmospherics', 'Sirius Catering',
       'Sirius Corporation', 'Sirius Drives', 'Sirius Hot2Cold',
       'Sirius Hyperspace', 'Sirius Industrial',
       'Sirius Luxury Transports', 'Sirius Mining Merope',
       'Sirius Mining', 'Sirius Power', 'The Greenventure Group',
       'The Peterson Group', 'The Rockforth Corporation',
       'Turner Research Group', 'Wiggins Development Trust',
       'Worster Insurance', 'Wreaken Construction', 'Bill Turner',
       'Aegis Defense']

In [16]:
a = csa['faction'].unique().tolist()
b = baseFrame['factionName'].unique().tolist()
aa = set(a)
bb = set(b)
bb-aa

set()

In [17]:
dates = []
inf = []

faction = "Combined Sirius Factions"
faction1 = "Sirius"

start_date = date(2018, 5, 15)
daily_date = date(2020,5,30)
end_date = date.today() + timedelta(days=1) #date(2020, 6, 28)


dframe = csa[ (csa['faction'].str.contains(faction1)) ]
regionNames = list(RegionFactory.regionDict.keys())
regionName = regionNames[region]
title = f"Total Influence of the {faction} Faction"

single_date = start_date
while single_date <= end_date:
    theDate = single_date.strftime("%Y-%m-%d")
    dated = dframe [ dframe['updated']<= theDate]

    stage1 = dated.loc[dated.groupby(['faction','system']).updated.idxmax()]
    stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
    theInf = stage3.influence.sum()
    dates.append(theDate)
    inf.append(theInf)
    if(single_date<daily_date):
      single_date = single_date + relativedelta(months=1)
    else:
      single_date = single_date + relativedelta(days=1)


labels = {'x':'Day', 'y':"Total Influence"}
fig = px.line(x=dates, y=inf, title=title, labels=labels)
fig.show()

In [18]:
# One System, handles Multiple Factions
fig = getSystemInfluenceGraph("Celaeno")
fig.show()

In [19]:
fig = getSystemInfluenceGraph("Solati")
fig.show()

In [20]:
# One System, handles Multiple Factions
fig = getSystemInfluenceGraph("Sol")
fig.show()

In [21]:
fac = "Hodack Prison Colony"
ogun = target[ target['faction']==fac]
a = ogun['system'].unique().tolist()
fig = go.Figure()
for sys in a:
  theSlice = ogun[ ogun['system'] == sys].reset_index()
  fig.add_trace(go.Scatter(x=theSlice['updated'], y=theSlice['influence'],
                    mode='lines',
                    name=sys,
                    stackgroup="one"))

getScroller(fig)
fig.show()


In [22]:
getFactionInfluenceGraph('Hodack Prison Colony').show()

In [23]:
fac = 'Sirius Mining'
fac = 'Sirius Mining Merope'
fac = 'CQC Holdings'
getFactionInfluenceGraph(fac).show()

In [24]:
from typing import List
def getCombinedFactionGraph( factions: List[str], titleSubject=None):
  labels = {'x':'Day', 'y':"Total Influence"}
  fig = go.Figure()
  for fac in factions:
    foo = target[target['faction']== fac]
    stage3 = foo[['system','faction','influence','updated']].groupby(['updated']).influence.sum().reset_index()
    fig.add_trace(go.Scatter(x=stage3['updated'], y=stage3['influence'],
                    mode='lines',
                    stackgroup="b",
                    name=fac))

  if titleSubject is None:
    titleSubject = "+".join(factions)

  title = f"Total influence of {titleSubject}"
  fig.update_layout(title=title,
                    xaxis_title="Date",
                    yaxis_title="Total Influence")
  getScroller(fig)
  return fig



In [25]:
faction1 = "Wreaken Construction"
faction2 = "Hodack Prison Colony"
factions = [faction1, faction2]

fig = getCombinedFactionGraph( factions, titleSubject=None)
fig.show()

In [26]:
siriusFactions = ['Sirius Atmospherics', 'Sirius Catering',
       'Sirius Corporation', 'Sirius Drives', 'Sirius Hot2Cold',
       'Sirius Hyperspace', 'Sirius Industrial',
       'Sirius Luxury Transports', 'Sirius Mining Merope',
       'Sirius Mining', 'Sirius Power']
      
fig = getCombinedFactionGraph( siriusFactions, titleSubject="Combined Sirius Factions")
fig.show()

In [81]:
csa = hist.getRawDataFrame()[['system','faction','updated','influence']]
display(csa)

AttributeError: ignored

                          system                faction    updated   influence
0                         Abroin   Abroin Universal PLC 2018-07-01   49.849500
1                         Abroin   Abroin Universal PLC 2018-08-01   45.600000
2                         Abroin   Abroin Universal PLC 2018-09-01   46.750567
3                         Abroin   Abroin Universal PLC 2018-10-01   59.000000
4                         Abroin   Abroin Universal PLC 2018-11-01   54.700000
...                          ...                    ...        ...         ...
10335  California Sector BV-Y c7  Turner Research Group 2020-07-31   12.900000
10336         Synuefai EB-R c7-5  Turner Research Group 2020-07-31  100.000000
10337    Mel 22 Sector GM-V c2-8  Turner Research Group 2020-07-31  100.000000
10338                     Delphi     Janus Incorporated 2020-07-31    8.316600
10339                     Delphi         Aegis Research 2020-07-31   10.220400

[10340 rows x 4 columns]

In [62]:
csa.columns

Index(['system', 'faction', 'updated', 'influence'], dtype='object')

In [46]:
#csa = csa.rename(columns={'system':'string', 'faction':'string', 'updated':'datetime', 'influence':'float', 'control':'boolean', 'region':'int','population':'float'})
csa = csa.rename(columns={'int':'id',
                          'string':'system',
                          'string':'faction',
                          'datetime': 'updated', 
                          'float':'influence',
                          'boolean':'control',
                          'int':'region',
                          'float': 'population'})

In [66]:
csa

AttributeError: ignored

                          system                faction    updated   influence
0                         Abroin   Abroin Universal PLC 2018-07-01   49.849500
1                         Abroin   Abroin Universal PLC 2018-08-01   45.600000
2                         Abroin   Abroin Universal PLC 2018-09-01   46.750567
3                         Abroin   Abroin Universal PLC 2018-10-01   59.000000
4                         Abroin   Abroin Universal PLC 2018-11-01   54.700000
...                          ...                    ...        ...         ...
10335  California Sector BV-Y c7  Turner Research Group 2020-07-31   12.900000
10336         Synuefai EB-R c7-5  Turner Research Group 2020-07-31  100.000000
10337    Mel 22 Sector GM-V c2-8  Turner Research Group 2020-07-31  100.000000
10338                     Delphi     Janus Incorporated 2020-07-31    8.316600
10339                     Delphi         Aegis Research 2020-07-31   10.220400

[10340 rows x 4 columns]

In [65]:
csa['system'].replace('', np.nan, inplace=True)
csa['faction'].replace('', np.nan, inplace=True)
csa['updated'].replace('', np.nan, inplace=True)
csa['influence'].replace('', np.nan, inplace=True)

In [52]:
pd.set_option('mode.chained_assignment', None)


In [82]:
from craid.eddb.loader.strategy.AWSLoader import LoadDataFromAWS
loader = LoadDataFromAWS(forceWebDownload=False, useSmol=False)
fName = loader.find_data_file('history.jsonl')
dataframe = pd.read_json(fName, lines=True, compression='infer')


Found data file: /tmp/history.jsonl.gz


In [83]:
dataframe.attrs

{}

In [84]:
dataframe = dataframe.rename(columns={'string':'system',
                          'string':'faction',
                          'datetime': 'updated', 
                          'float':'influence',
                          'boolean':'control',
                          'int':'region',
                          'float': 'population'})

In [85]:
dataframe

AttributeError: ignored

                          system                faction  ...  region  population
0                         Abroin   Abroin Universal PLC  ...       6           0
1                         Abroin   Abroin Universal PLC  ...       6           0
2                         Abroin   Abroin Universal PLC  ...       6           0
3                         Abroin   Abroin Universal PLC  ...       6           0
4                         Abroin   Abroin Universal PLC  ...       6           0
...                          ...                    ...  ...     ...         ...
10335  California Sector BV-Y c7  Turner Research Group  ...       3        5000
10336         Synuefai EB-R c7-5  Turner Research Group  ...       3       15000
10337    Mel 22 Sector GM-V c2-8  Turner Research Group  ...       0       18550
10338                     Delphi     Janus Incorporated  ...       1      300000
10339                     Delphi         Aegis Research  ...       1      300000

[10340 rows x 7 columns]